In [1]:
using Distributions
using ReactiveMP
using Rx
using BenchmarkTools

import Base: show

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1273
┌ Warning: Package ReactiveMP does not have Rx in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rx as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rx into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [2]:
mutable struct InferenceActor <: Actor{AbstractMessage}
    index       :: Int
    size        :: Int
    data        :: Vector{Float64}
    messages    :: Vector{AbstractMessage}
    communicate :: Channel{Tuple{Float64, Float64}}
    
    y          :: ObservedVariable
    e_mean     :: EstimatedVariable
    e_variance :: EstimatedVariable
    
    InferenceActor(data::Vector{Float64}, y::ObservedVariable, e_mean::EstimatedVariable, e_variance::EstimatedVariable) = begin
        size  = length(data)
        messages = Vector{AbstractMessage}(undef, size)
        
        actor = new(1, size, data, messages, Channel{Tuple{Float64, Float64}}(Inf), y, e_mean, e_variance)
        
        task = @async begin
            while true
                u = take!(actor.communicate)
                update!(actor, u[1], u[2])
            end
        end
        
        bind(actor.communicate, task)
        
        return actor
    end
end

function update!(actor::InferenceActor, mean::Float64, variance::Float64)
    next!(actor.y.values, actor.data[actor.index])
    next!(actor.e_mean.values, mean)
    next!(actor.e_variance.values, variance)
end

function stop!(actor::InferenceActor)
    complete!(actor.y.values)
    complete!(actor.e_mean.values)
    complete!(actor.e_variance.values)
end

function Rx.on_next!(actor::InferenceActor, data::AbstractMessage)
    m = mean(data.distribution)
    v = var(data.distribution)
    
    actor.messages[actor.index] = data
    
    actor.index += 1
    
    if actor.index < actor.size + 1
        put!(actor.communicate, (m, v))
    else
        stop!(actor)
    end
end

Rx.on_error!(actor::InferenceActor, err) = error(err)

Rx.on_complete!(actor::InferenceActor)   = begin 
    close(actor.communicate)
end

Base.show(io::IO, actor::InferenceActor) = print(io, "InferenceActor")

In [3]:
function kalman()
    N = 5000
    data = collect(1:N) + sqrt(200.0) * randn(N);
    
    x_prev_add   = AdditionNode("x_prev_add");
    add_1        = ConstantVariable("add_1", 1.0, x_prev_add.in2);

    x_prev_prior = GaussianMeanVarianceNode("x_prev_prior");
    x_prev_m     = EstimatedVariable("x_prev_m", x_prev_prior.mean);
    x_prev_v     = EstimatedVariable("x_prev_v", x_prev_prior.variance);

    x_prev = RandomVariable("x_prev", x_prev_prior.value, x_prev_add.in1);

    noise_node     = GaussianMeanVarianceNode("noise_node");
    noise_mean     = ConstantVariable("noise_mean", 0.0, noise_node.mean);
    noise_variance = ConstantVariable("noise_variance", 200.0, noise_node.variance);

    add_x_and_noise = AdditionNode("add_x_and_noise");

    x = RandomVariable("x", x_prev_add.out, add_x_and_noise.in1);
    n = RandomVariable("n", noise_node.value, add_x_and_noise.in2);
    y = ObservedVariable("y", add_x_and_noise.out);
    
    actor  = InferenceActor(data, y, x_prev_m, x_prev_v);
    synced = sync(actor)
    
    @async begin
        try
            subscribe!(inference(x), synced)   
            update!(actor, 0.0, 1000.0) 
        catch e 
            println(e)
        end
    end
    
    wait(synced)
    
    return actor.messages
end

kalman (generic function with 1 method)

In [7]:
@btime kalman();

  9.612 ms (190426 allocations: 7.30 MiB)


In [6]:
@time kalman();

  0.014066 seconds (190.43 k allocations: 7.304 MiB)
